<a href="https://colab.research.google.com/github/heartzhacker/Emotion-classification/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import zipfile

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
local_data_folder = "/content/Data"
os.makedirs(local_data_folder, exist_ok=True)

In [8]:
data_zip_file = "/content/drive/MyDrive/data.zip"

with zipfile.ZipFile(data_zip_file, 'r') as zip_ref:
    zip_ref.extractall(local_data_folder)

In [9]:
!pip install deepface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=7b6ba4dcdf58ad5dc364fe20af1e43e496cf666c73d538d8dc1e5a37a50d27ac
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [10]:
from deepface import DeepFace
def classify_gender(frame):
    result = DeepFace.analyze(frame, actions=['gender'])
    return result['gender']

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [11]:
def extract_info(video_file):
    parts = video_file.split("-")
    emotion = int(parts[2])
    intensity = int(parts[3])
    statement = int(parts[4])
    gender = "Man" if int(parts[6]) % 2 == 0 else "Woman"
    return emotion, intensity, statement, gender

In [12]:
def load_videos_and_info(dataset_path):
    videos = []
    emotions = []
    intensities = []
    statements = []
    genders = []

    for actor_folder in os.listdir(dataset_path):
        actor_path = os.path.join(dataset_path, actor_folder)
        if os.path.isdir(actor_path):
            for video_file in os.listdir(actor_path):
                video_path = os.path.join(actor_path, video_file)
                if video_path.endswith(".mp4"):
                    video = cv2.VideoCapture(video_path)
                    videos.append(video)
                    emotion, intensity, statement, gender = extract_info(video_file)
                    emotions.append(emotion)
                    intensities.append(intensity)
                    statements.append(statement)
                    genders.append(gender)

    return videos, emotions, intensities, statements, genders

videos, emotions, intensities, statements, genders = load_videos_and_info(local_data_folder)

In [13]:
male_videos = []
female_videos = []
male_emotions = []
female_emotions = []
male_intensities = []
female_intensities = []
male_statements = []
female_statements = []

for video, emotion, intensity, statement, gender in zip(videos, emotions, intensities, statements, genders):
    if gender == 'Man':
        male_videos.append(video)
        male_emotions.append(emotion)
        male_intensities.append(intensity)
        male_statements.append(statement)
    elif gender == 'Woman':
        female_videos.append(video)
        female_emotions.append(emotion)
        female_intensities.append(intensity)
        female_statements.append(statement)

In [15]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 shape_predictor_68_face_landmarks.dat.bz2

--2023-06-10 20:19:57--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  89.9MB/s    in 0.7s    

2023-06-10 20:19:58 (89.9 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [16]:
import dlib
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb

def preprocess_video(video, face_detector, face_aligner, target_size=(224, 224)):
    ret, frame = video.read()
    if not ret:
        return None

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)

    if len(faces) == 0:
        return None

    face = faces[0]
    (x, y, w, h) = rect_to_bb(face)
    face_aligned = face_aligner.align(frame, gray, face)
    face_resized = cv2.resize(face_aligned, target_size)

    return face_resized

# Initialize face detector and face aligner
face_detector = dlib.get_frontal_face_detector()
face_aligner = FaceAligner(dlib.shape_predictor("shape_predictor_68_face_landmarks.dat"))

# Preprocess male videos
male_faces = [preprocess_video(video, face_detector, face_aligner) for video in male_videos]

# Preprocess female videos
female_faces = [preprocess_video(video, face_detector, face_aligner) for video in female_videos]